In [1]:
''' 
 RUSBOOST BINARY
'''
import pandas as pd
def FoundFeatures(Path):
    # reading the found dataset for the training
    df = pd.read_excel(Path)
    Features = set(df.columns)
    return Features

# removing the id and the subtypes of a given dataset
def extract_PATIENT_INFO(db):
    ids =  db.pop("ID")
    morf_codificata = db.pop("morf_codificata")
    luogoTC = db.pop("luogoTC_codificato")

    return db, ids, morf_codificata, luogoTC

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def saving_importances(feature_importances, X, split_iter):
    feature_names = X.columns
    # Sort feature importances and names
    sorted_indices = np.argsort(feature_importances)[::-1]
    sorted_importances = feature_importances[sorted_indices]
    sorted_features = np.array(feature_names)[sorted_indices]

    # Select only the top 10 features
    top_n = 10
    sorted_importances = sorted_importances[:top_n]
    sorted_features = sorted_features[:top_n]

    # Set style
    sns.set_theme(style="whitegrid")  # Use a Seaborn style for a cleaner look

    # Plot
    plt.figure(figsize=(10, 6))  # Adjust the size as needed
    bars = plt.bar(sorted_features, sorted_importances, color=sns.color_palette("autumn", len(sorted_importances)), edgecolor='black')

    # Add annotations
    for bar, importance in zip(bars, sorted_importances):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            f"{importance:.2f}",
            ha='center',
            va='bottom',
            fontsize=10,
            color='black',
        )

    plt.xticks(rotation=45, ha='right', fontsize=10)  # Reduced fontsize for feature names
    plt.yticks(fontsize=12)  # Keep y-axis labels at a larger size

    # Set the y-axis limit (adjust the upper limit for better spacing)
    plt.ylim(0, max(sorted_importances) + 0.1)  # You can adjust the padding (0.1) as needed
    plt.xlabel('Feature Names', fontsize=14, labelpad=10)
    plt.ylabel('Importance Score', fontsize=14, labelpad=10)
    plt.title(f'Top {top_n} Feature Importances from RUSBoostClassifier (Split {split_iter})', fontsize=16, pad=15)
    plt.tight_layout()

    # Save the plot
    plt.savefig(f'Elements/RusBoost/FirstOrder/Split{split_iter}_top_{top_n}_feature_importances.png', dpi=300, bbox_inches='tight')
    plt.close()



In [3]:
from Components import Ingestion
db = Ingestion.extractFirstOrder("Dataset/db_29-07-2024_2_Ing_Petti.xlsx")
db.shape

[2024-12-02 14:25:09,307: INFO: Ingestion: from first fitering using the tenere_finale attribute,  I obtained (1108, 184) patients]
[2024-12-02 14:25:09,309: INFO: Ingestion: from removing columns with too many Nans,I obtained (1046, 128) patients]
['ID', 'morf_codificata', 'maligno', 'luogoTC_codificato', 'BAS_INTENSITYBASED_MeanHUIBSIQ4LE', 'BAS_INTENSITYBASED_VarianceHUIBSIECT3', 'BAS_INTENSITYBASED_SkewnessHUIBSIKE2A', 'BAS_INTENSITYBASED_KurtosisHUIBSIIPH6', 'BAS_INTENSITYBASED_MedianHUIBSIY12H', 'BAS_INTENSITYBASED_MinimumGreyLevelHUIBSI1GSF', 'BAS_INTENSITYBASED_10thPercentileHUIBSIQG58', 'BAS_INTENSITYBASED_25thPercentileHUIBSINo', 'BAS_INTENSITYBASED_50thPercentileHUIBSIY12H', 'BAS_INTENSITYBASED_75thPercentileHUIBSINo', 'BAS_INTENSITYBASED_90thPercentileHUIBSI8DWT', 'BAS_INTENSITYBASED_StandardDeviationHUIBSINo', 'BAS_INTENSITYBASED_MaximumGreyLevelHUIBSI84IY', 'BAS_INTENSITYBASED_InterquartileRangeHUIBSISALO', 'BAS_INTENSITYBASED_RangeHUIBSI2OJQ', 'BAS_INTENSITYBASED_MeanAbs

(1046, 58)

In [4]:

db = pd.read_excel("Dataset/OnlyFirstOrder.xlsx")
db.shape

(1046, 58)

In [5]:
db.head()

,ID,morf_codificata,maligno,luogoTC_codificato,BAS_INTENSITYBASED_MeanHUIBSIQ4LE,BAS_INTENSITYBASED_VarianceHUIBSIECT3,BAS_INTENSITYBASED_SkewnessHUIBSIKE2A,BAS_INTENSITYBASED_KurtosisHUIBSIIPH6,BAS_INTENSITYBASED_MedianHUIBSIY12H,BAS_INTENSITYBASED_MinimumGreyLevelHUIBSI1GSF,...,BAS_INTENSITYHISTOGRAM_IntensityHistogramQuartileCoefficientOfDi,BAS_INTENSITYHISTOGRAM_IntensityHistogramEntropyLog10HUIBSINo,BAS_INTENSITYHISTOGRAM_IntensityHistogramEntropyLog2HUIBSITLU2,BAS_INTENSITYHISTOGRAM_AreaUnderCurveCshHUIBSINo,BAS_INTENSITYHISTOGRAM_UniformityHUIBSIBJ5W,BAS_INTENSITYHISTOGRAM_RootMeanSquareHUIBSINo,BAS_INTENSITYHISTOGRAM_MaximumHistogramGradientHUIBSI12CE,BAS_INTENSITYHISTOGRAM_MaximumHistogramGradientGreyLevelHUIBSI8E,BAS_INTENSITYHISTOGRAM_MinimumHistogramGradientHUIBSIVQB3,BAS_INTENSITYHISTOGRAM_MinimumHistogramGradientGreyLevelHUIBSIRH
0,RADIO-BOL_1,0,0,1,-3.212463,394.202158,-0.613901,0.781620,-1.227348,-71.489891,...,0.014925,0.897388,2.981060,1001.884058,0.155167,0.016766,41.5,99,-42.0,102
1,RADIO-BOL_2,0,0,1,-22.545812,638.736874,-0.287418,0.156812,-21.938194,-113.263260,...,0.015228,1.017221,3.379136,982.571429,0.112154,0.010335,38.5,96,-40.0,102
2,RADIO-BOL_3,1,0,1,24.612552,299.947559,-0.014479,0.088602,24.338148,-37.693333,...,0.009709,0.858563,2.852083,1029.664179,0.162049,0.009294,142.0,101,-139.0,105
3,RADIO-BOL_4,0,0,1,-26.732099,471.036614,-0.652063,0.869541,-23.557526,-117.619003,...,0.010204,0.936254,3.110169,978.260870,0.144627,0.005119,341.5,97,-476.5,100
4,RADIO-BOL_6,1,0,1,11.925845,718.019469,-1.005982,2.719904,13.872608,-144.765686,...,0.014778,1.018064,3.381935,1016.891754,0.116761,0.004451,240.5,100,-252.5,105


In [6]:
OLD = pd.read_excel("Dataset/Cleaned_dataset.xlsx")
OLD.shape

(1046, 128)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
import numpy as np

# create dataset
db = pd.read_excel("Dataset/OnlyFirstOrder.xlsx")

# extrating the label
y = db.pop("maligno")
X = db

X_train_folds = []
X_test_folds = []

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
best_params_per_fold = []
scores = pd.DataFrame(columns=['Outer Fold',"Grid-Search",'F1-score', 'Recall', 'Precision', 'Accuracy', 'Auc-Score', '#Features'])

for index, (train_ix, test_ix) in enumerate(cv_outer.split(X, y)):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    

    unique, counts = np.unique(y_test, return_counts=True)
    hist = dict(zip(unique, counts))
    print(f"Fold {index} - Test set class distribution: {hist}")
    X_test_outer = X_test
    X_test_folds.append(X_test)
    morf = X_test_outer["morf_codificata"]
    unique, counts = np.unique(morf, return_counts=True)
    hist = dict(zip(unique, counts))
    print(f"Fold {index} - Test set class distribution: {hist}")


    fold_df = pd.DataFrame(X_test)
    fold_df["label"] = y_test
    fold_df.to_csv(f"nested_CV/FirstOrder/fold_{index}_test_set.csv", index=False)


    fold_df = pd.DataFrame(X_train)
    X_train_folds.append(fold_df)
    fold_df["label"] = y_train
    fold_df.to_csv(f"nested_CV/FirstOrder/fold_{index}_train_set.csv", index=False)


Fold 0 - Test set class distribution: {0: 168, 1: 42}
Fold 0 - Test set class distribution: {0: 108, 1: 56, 2: 23, 3: 15, 4: 4, 5: 4}
Fold 1 - Test set class distribution: {0: 168, 1: 41}
Fold 1 - Test set class distribution: {0: 116, 1: 46, 2: 21, 3: 8, 4: 12, 5: 6}
Fold 2 - Test set class distribution: {0: 167, 1: 42}
Fold 2 - Test set class distribution: {0: 134, 1: 29, 2: 16, 3: 14, 4: 12, 5: 4}
Fold 3 - Test set class distribution: {0: 167, 1: 42}
Fold 3 - Test set class distribution: {0: 118, 1: 46, 2: 18, 3: 13, 4: 11, 5: 3}
Fold 4 - Test set class distribution: {0: 167, 1: 42}
Fold 4 - Test set class distribution: {0: 114, 1: 51, 2: 22, 3: 10, 4: 10, 5: 2}


In [8]:
from numpy import mean
from numpy import std
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from Models import RusBoost
from sklearn.metrics import  accuracy_score, confusion_matrix,precision_score, recall_score, f1_score,roc_auc_score,roc_curve,precision_recall_curve, average_precision_score



db = pd.read_excel("Dataset/OnlyFirstOrder.xlsx")


 # extrating the label
y = db.pop("maligno")
X = db

# configure the cross-validation procedure
cv_outer = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)
# enumerate splits
outer_results = list()
best_params_per_fold = []
scores = pd.DataFrame(columns=['Outer Fold',"Grid-Search",'F1-score', 'Recall', 'Precision', 'Accuracy', 'Auc-Score', '#Features'])


folds_train = []
folds_test = []

for index, (train_ix, test_ix) in enumerate(cv_outer.split(X, y)):
	# split data
	X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
	y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]


	folds_train.append(X_train.copy())
	folds_test.append(X_test.copy())

	# configure the cross-validation procedure
	cv_inner = StratifiedKFold(n_splits=5,shuffle = True, random_state=42)
	
	pipeline = RusBoost.Define_Pipeline()
	X_train, ids, morf_codificata, luogoTC_train = extract_PATIENT_INFO(X_train)
	Grid_searchOBJ = RusBoost.Grid_Search(pipeline=pipeline, cv = cv_inner, X_train=X_train, y_train=y_train)

	best_params = Grid_searchOBJ.best_params_
	best_params_per_fold.append(Grid_searchOBJ.best_params_)
	print(f"Fold {len(best_params_per_fold)}: Best Params = {Grid_searchOBJ.best_params_}")

	Chosen_pipeline = RusBoost.Define_Pipeline()
	Chosen_pipeline.set_params(**Grid_searchOBJ.best_params_) 
	fitted_pipeline = RusBoost.fit(pipeline=Chosen_pipeline, X_train=X_train, y_train=y_train, Prefix="FirstOrder/Split{}_".format(index))


	feats_import = fitted_pipeline["rusboostclassifier"].feature_importances_
	saving_importances(feats_import, X_train, index)

	X_test, ids_test, morf_codificata_test, luogoTC_test  = extract_PATIENT_INFO(X_test)
	predictions = fitted_pipeline.predict(X_test)
	probs = fitted_pipeline.predict_proba(X_test)

	###Metrics
	Accuracy = round(accuracy_score(y_test, predictions),3)
	precision = round(precision_score(y_test, predictions),3)
	recall = round(recall_score(y_test, predictions),3)
	f1 = round(f1_score(y_test, predictions),3)
	print("F1 Test is", f1)
	probs = probs[:, 1]
	auc_roc = round(roc_auc_score(y_test,probs),3)

	##Features
	AllFeatures_RusBoost = FoundFeatures("./Elements/RusBoost/FirstOrder/Split{}_RusBoost_ReduceDataset.xlsx".format(index))
	print("it has found {} features".format(len(AllFeatures_RusBoost)))
	
	new_row = [index,Grid_searchOBJ.best_score_, f1, recall, precision, Accuracy, auc_roc, len(AllFeatures_RusBoost)]  
	scores.loc[index] = new_row  # Append the new row
    
scores["Outer Fold"] = scores["Outer Fold"].astype(int)
scores["#Features"] = scores["#Features"].astype(int)
scores.to_csv('Elements/RusBoost/FirstOrder/scores.csv', index=False)  # index=False to avoid saving the row numbers
scores

[2024-12-02 14:25:12,469: INFO: RusBoost: Defining the pipeline...]
[2024-12-02 14:25:12,470: INFO: RusBoost: Let's define the cross validation:StratifiedKFold(n_splits=5, random_state=42, shuffle=True)]
[2024-12-02 14:25:12,471: INFO: RusBoost: Let's define the parameters for the grid search: {'rusboostclassifier__n_estimators': [150, 300], 'rusboostclassifier__learning_rate': [0.05, 0.01]}]
Best: 0.807345 using {'rusboostclassifier__learning_rate': 0.05, 'rusboostclassifier__n_estimators': 150}
Fold 1: Best Params = {'rusboostclassifier__learning_rate': 0.05, 'rusboostclassifier__n_estimators': 150}
[2024-12-02 14:25:18,107: INFO: RusBoost: Defining the pipeline...]
Time taken to fit the model: 0.45371 seconds
[2024-12-02 14:25:19,188: INFO: RusBoost: saving the resulting dataframe into: RusBoost_ReduceDataset.xlsx]
F1 Test is 0.755
it has found 54 features
[2024-12-02 14:25:20,016: INFO: RusBoost: Defining the pipeline...]
[2024-12-02 14:25:20,017: INFO: RusBoost: Let's define the c

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.807345,0.755,0.881,0.661,0.886,0.964,54
1,1,0.778075,0.796,0.951,0.684,0.904,0.977,54
2,2,0.787657,0.841,0.881,0.804,0.933,0.968,54
3,3,0.805012,0.771,0.881,0.685,0.895,0.964,54
4,4,0.795484,0.800,0.952,0.690,0.904,0.964,54


In [9]:
scores
	

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.807345,0.755,0.881,0.661,0.886,0.964,54
1,1,0.778075,0.796,0.951,0.684,0.904,0.977,54
2,2,0.787657,0.841,0.881,0.804,0.933,0.968,54
3,3,0.805012,0.771,0.881,0.685,0.895,0.964,54
4,4,0.795484,0.800,0.952,0.690,0.904,0.964,54


In [10]:
best_params_per_fold 

[{'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 150},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 150},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 150},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 150},
 {'rusboostclassifier__learning_rate': 0.05,
  'rusboostclassifier__n_estimators': 300}]

In [11]:
import numpy as np
def compute_stats(scores):
    cols = scores.columns[1:-1]
    mean_row_values = scores[cols].mean()
    std_row_values = scores[cols].std()
    
    # Create new rows with NaNs for the columns that will be replaced
    mean_row = pd.Series("", index=scores.columns, name='mean')
    std_row = pd.Series("", index=scores.columns, name='std')
    mean_row[cols] = mean_row_values.round(3)
    std_row[cols] = std_row_values.round(3)
    
    # Convert Series to DataFrame
    mean_df = mean_row.to_frame().T
    std_df = std_row.to_frame().T
    
    scores_new = pd.concat([scores, mean_df, std_df], axis = 0)
    
    scores_new.loc["mean","#Features"] = str(int(round(np.mean(scores_new["#Features"][:-2]),0)))
    scores_new.loc["std","#Features"] = str(int(round(np.std( scores_new["#Features"][:-2]),0)))
    
    scores_new.to_csv('Elements/RusBoost/FirstOrder/scores.csv', index=False)  # index=False to avoid saving the row numbers

    return scores_new

scores_agg = compute_stats(scores)
scores_agg

,Outer Fold,Grid-Search,F1-score,Recall,Precision,Accuracy,Auc-Score,#Features
0,0,0.807345,0.755,0.881,0.661,0.886,0.964,54
1,1,0.778075,0.796,0.951,0.684,0.904,0.977,54
2,2,0.787657,0.841,0.881,0.804,0.933,0.968,54
3,3,0.805012,0.771,0.881,0.685,0.895,0.964,54
4,4,0.795484,0.8,0.952,0.69,0.904,0.964,54
mean,,0.795,0.793,0.909,0.705,0.904,0.967,54
std,,0.012,0.033,0.039,0.057,0.018,0.006,0
